# Парсинг сайта vidal.ru

In [1]:
from bs4 import BeautifulSoup
import requests as req
from  more_itertools import unique_everseen
from tqdm import tqdm
from tqdm import tqdm_notebook as tq
from general_func import hascyr, recursive_flatten_generator, list_to_file, read_list
import warnings
warnings.filterwarnings('ignore')
import codecs
import re

#### Названия препаратов

In [2]:
start_ref = req.get("https://www.vidal.ru/drugs/products")

In [3]:
soup = BeautifulSoup(start_ref.text, 'lxml')
#soup = soup.prettify()

In [4]:
#Список ссылок на препараты
letter_links = soup.find("div", attrs={ "class" : "letters-russian"})
l_links = []
for link in letter_links.find_all('a'):
    l_links.append(link.get('href'))
letter_pages = ['https://www.vidal.ru'+link for link in l_links]
full_pages=[]
for ind in range(2,25):
    for page in letter_pages:
        p = page+'?p=%d"' %ind 
        full_pages.append(p)
full_pages.append(letter_pages)
full_pages = recursive_flatten_generator(full_pages)

In [5]:
full_pages[:10]

['https://www.vidal.ru/drugs/products/p/rus-a?p=2"',
 'https://www.vidal.ru/drugs/products/p/rus-b?p=2"',
 'https://www.vidal.ru/drugs/products/p/rus-v?p=2"',
 'https://www.vidal.ru/drugs/products/p/rus-g?p=2"',
 'https://www.vidal.ru/drugs/products/p/rus-d?p=2"',
 'https://www.vidal.ru/drugs/products/p/rus-e?p=2"',
 'https://www.vidal.ru/drugs/products/p/rus-zh?p=2"',
 'https://www.vidal.ru/drugs/products/p/rus-z?p=2"',
 'https://www.vidal.ru/drugs/products/p/rus-i?p=2"',
 'https://www.vidal.ru/drugs/products/p/rus-j?p=2"']

In [5]:
full_links_list = []
full_links=[]
l=[]
p=[]
for page in tq(full_pages):
    ref = req.get(page)
    soup = BeautifulSoup(ref.text, 'lxml')
    links = soup.find_all("a", attrs={"class" : "no-underline"})
    full_links.append(links)
full_links = recursive_flatten_generator(full_links)

In [10]:
#full_href = read_list('vidal_links.txt', 'utf-8')
#full_href = list(unique_everseen(full_href))
#len(full_href)

23431

In [8]:
#Исключаем лишние позиции
for item in full_links:
    if not 'firm' in item.get('href'):
        l.append(item)
for item in l:
    if not 'company' in item.get('href'):
        p.append(item)
for item in p:
    if not 'password' in item.get('href'):
        full_links_list.append(item)
full_links_list[:5]

[<a class="no-underline" href="/drugs/adaclean__31361">
                 Адаклин
             </a>,
 <a class="no-underline" href="/drugs/adam__35144">
                 Адам<sup>®</sup>
 </a>,
 <a class="no-underline" href="/drugs/adam__35144"></a>,
 <a class="no-underline" href="/drugs/adaptol__11182">
                 Адаптол<sup>®</sup>
 </a>,
 <a class="no-underline" href="/drugs/adaptol__11182"></a>]

In [10]:
med_names = [item.get_text() for item in full_links_list]
med_names = list(unique_everseen(med_names))
medications = [item.replace('\n                ','').replace('\n','').replace('®','').replace('            ','') for item in med_names]
#list_to_file('drug_names.txt', sorted(medications), 'cp1251')
medications[:10]

['Адаклин',
 'Адам',
 '',
 'Адаптол',
 'Адасель',
 'Адваграф',
 'Адвантан',
 'Адвейт',
 'Адгелон',
 'Аддамель Н']

In [11]:
href = [item.get('href') for item in tq(full_links_list)]
full_href = ['https://www.vidal.ru'+item for item in tq(href)]
full_href = list(unique_everseen(full_href))

In [14]:
#Сохраняем список всех ссылок на лекарственные препараты
#list_to_file('vidal_links.txt', full_href, 'utf-8')

In [12]:
medications = med_names
medications = [item.lower() for item in medications]

In [13]:
#Лучше использовать из сохраненного файла, парсится долго (час-два)
# Файл 'names_trans_1.txt'
names=[]
for page in tq(full_href):
    ref = req.get(page)
    soup = BeautifulSoup(ref.text, 'lxml')
    rus_name = soup.find_all("span", attrs={ "class" : "anons-product"})
    eng_name = soup.find_all("span", attrs={ "class" : "atc-name small"}) 
    rus = [item.get_text().lower() for item in rus_name]
    eng = [item.get_text().lower() for item in eng_name]
    names.append((rus,eng))

In [17]:
#Список название препарата - действующее вещество на английском языке, далее используется при сопоставленипи кодов SNOMED CT
names[:10]

[(['адаклин'], ['(adapalene)']),
 (['адам®'], ['(не присвоен)']),
 (['адаптол®'], ['(tetramethylglycoluril)']),
 (['адасель'], ['(pertussis, purified antigen, combinations with toxoids)']),
 (['адваграф®'], ['(tacrolimus)']),
 (['адваграф®'], ['(tacrolimus)']),
 (['адваграф®'], ['(tacrolimus)']),
 (['адваграф®'], ['(tacrolimus)']),
 (['адвантан®'], ['(methylprednisolone aceponate)']),
 (['адвейт®'], ['(coagulation factor viii)'])]

In [24]:
import pandas as pd
import numpy as np

#rus = [name[0] for name in names]
#eng = [name[1] for name in names]
#df = pd.DataFrame({'rus' : []})
#df = pd.DataFrame({'rus' : []})
#df['rus'] = rus
#df['eng'] = eng
#df_in = df[df[0].isin(medications)]
#df.to_csv(r'names_trans_1.txt', header=None, index=None, sep='\t', encoding='cp1251')

In [18]:
df = pd.read_csv('names_trans_1.txt', encoding='cp1251', sep='\t', engine='python', header=None, error_bad_lines=False)
df = df.drop_duplicates().reset_index()
df.head()

,index,0,1
0,0,аевит,multivitamin
1,1,валериана,valerianae radix
2,2,геделикс,Hederae helicis foliorum extract
3,3,гексорал,hexetidine
4,4,гелофузин,Gelatine


#### Действующие вещества

In [19]:
start_page = 'https://www.vidal.ru/drugs/molecules?p='
pages=[]
for ind in range(1,50):
    page = start_page+'%d' %ind
    pages.append(page)

In [20]:
#Список ссылок на действующие вещества
all_molec_links=[]
for page in tqdm(pages):
    ref = req.get(page)
    soup = BeautifulSoup(ref.text, 'lxml')
    table = soup.find("table", attrs={ "class" : "products-table molecules"})  
    molec_links = ['https://www.vidal.ru'+item.get('href') for item in table.find_all('a')]
    uniq_links = list(unique_everseen(molec_links))
    all_molec_links.append(uniq_links)
all_molec_links = recursive_flatten_generator(all_molec_links)
all_molec_links = sorted(list(unique_everseen((all_molec_links))))
all_molec_links = all_molec_links[1:]

100%|██████████| 49/49 [01:45<00:00,  2.16s/it]


In [30]:
all_molec_links[:10]

['https://www.vidal.ru/drugs/molecule/1',
 'https://www.vidal.ru/drugs/molecule/10',
 'https://www.vidal.ru/drugs/molecule/1002',
 'https://www.vidal.ru/drugs/molecule/1003',
 'https://www.vidal.ru/drugs/molecule/1004',
 'https://www.vidal.ru/drugs/molecule/1005',
 'https://www.vidal.ru/drugs/molecule/1006',
 'https://www.vidal.ru/drugs/molecule/1007',
 'https://www.vidal.ru/drugs/molecule/1008',
 'https://www.vidal.ru/drugs/molecule/1009']

In [32]:
#Парсинг русскоязычных названий действующих веществ (для поиска в текстах)
full_rus_name=[]
for link in tqdm(all_molec_links):
    ref = req.get(link)
    soup = BeautifulSoup(ref.text, 'lxml')
    name = soup.find("h1")
    rus_name = name.get_text()
    full_rus_name.append(rus_name)
full_rus_name = recursive_flatten_generator(full_rus_name)

100%|██████████| 2070/2070 [03:36<00:00,  9.55it/s]


In [33]:
full_rus_name[:10]

['        АБЦИКСИМАБ (ABCIXIMAB) ОПИСАНИЕ\n    ',
 '        АЦЕТИЛЦИСТЕИН (ACETYLCYSTEINE) ОПИСАНИЕ\n    ',
 '        СУКСАМЕТОНИЯ ХЛОРИД (SUXAMETHONIUM CHLORIDE) ОПИСАНИЕ\n    ',
 '        ТАЛИНОЛОЛ (TALINOLOL) ОПИСАНИЕ\n    ',
 '        ТАМОКСИФЕН (TAMOXIFEN) ОПИСАНИЕ\n    ',
 '        ТАМСУЛОЗИН (TAMSULOSIN) ОПИСАНИЕ\n    ',
 '        ЦВЕТКИ ПИЖМЫ ОБЫКНОВЕННОЙ (TANACETUM VULGARE FLORES) ОПИСАНИЕ\n    ',
 '        ДЕГОТЬ (TAR) ОПИСАНИЕ\n    ',
 '        КОРНИ И ТРАВА ОДУВАНЧИКА (TARAXACI RADIX CUM HERBA) ОПИСАНИЕ\n    ',
 '        ТАУРИН (TAURINE) ОПИСАНИЕ\n    ']

In [35]:
#list_to_file('molec_links.txt', sorted(all_molec_links),'cp1251')
#list_to_file('molec_names.txt', sorted(full_rus_name),'cp1251')

# Парсинг рос реестра лекартственных препаратов

In [2]:
main_page_str = "https://www.rlsnet.ru"
main_page = req.get(main_page_str)

start_page = req.get("https://www.rlsnet.ru/tn_alf_letter_c0.htm")
soup = BeautifulSoup(start_page.text, 'lxml')

#### Сбор страниц

In [3]:
def get_letter_pages(tag, attribute, attr_value):
    all_pages=[]
    letter_links = soup.find_all(tag, attrs={ attribute : attr_value})
    for link in letter_links:
        page = main_page_str+link.get('href')
        all_pages.append(page)
    return all_pages

def get_names(all_pages, tag):
    names=[]
    for page in tqdm_notebook(all_pages):
        ref = req.get(page)
        soup = BeautifulSoup(ref.text, 'lxml')
        name = soup.find_all(tag)
        rus_name = [item.get_text() for item in name]
        names.append(rus_name)
    return names

In [4]:
#Страницы в РЛС пронумерованы, получаем их генерацией
pages =[]
for ind in range(0,10):
    pages.append('https://www.rlsnet.ru/tn_alf_letter_c%i.htm?' %ind)
let = ['a','b','c','d','e','f']
for item in let:
    pages.append('https://www.rlsnet.ru/tn_alf_letter_c%s.htm?' %item)
for ind in range(0,9):
    pages.append('https://www.rlsnet.ru/tn_alf_letter_d%i.htm?' %ind)
for ind in let[-3:]:
    pages.append('https://www.rlsnet.ru/tn_alf_letter_d%s.htm?' %ind)
pages.append('https://www.rlsnet.ru/tn_alf_letter_1.htm?')
pages[:10]

['https://www.rlsnet.ru/tn_alf_letter_c0.htm?',
 'https://www.rlsnet.ru/tn_alf_letter_c1.htm?',
 'https://www.rlsnet.ru/tn_alf_letter_c2.htm?',
 'https://www.rlsnet.ru/tn_alf_letter_c3.htm?',
 'https://www.rlsnet.ru/tn_alf_letter_c4.htm?',
 'https://www.rlsnet.ru/tn_alf_letter_c5.htm?',
 'https://www.rlsnet.ru/tn_alf_letter_c6.htm?',
 'https://www.rlsnet.ru/tn_alf_letter_c7.htm?',
 'https://www.rlsnet.ru/tn_alf_letter_c8.htm?',
 'https://www.rlsnet.ru/tn_alf_letter_c9.htm?']

In [5]:
#Получаем общее количество страниц
links=[]
for link in tq(pages):
    page = req.get(link)
    soup = BeautifulSoup(page.text, 'lxml')
    content = soup.find_all('li')
    c = [item.find_all('a') for item in content]
    c = recursive_flatten_generator(c)
    for item in c:
        if not 'class' in str(item):
            link = item.get('href')
            links.append(link)
links = list(unique_everseen(links))
links=['http://'+item[6:] for item in links]
len(links)

14936

In [26]:
#Также парсится долго, лучше читать из файла
#Получаем имена
molec=[]
names=[]
for link in tq(links):
    page = req.get(link)
    soup = BeautifulSoup(page.text, 'lxml')
    a = soup.find_all('a', attrs={'class':'drug__link drug__link--article'})
    for item in a:
        molec.append(item.get_text())
    a = soup.find_all('h1')
    if a:
        names.append(a[0].get_text())


In [28]:
names[:10]

['А.Т.10® (A.T.10®)',
 'аАКДС-Геп В+Hib (Вакцина против дифтерии, столбняка, гепатита В, коклюша бесклеточная адсорбированная, инфекции, вызываемой Haemophilus influenzae тип b, конъюгированная синтетическая)',
 'Абаджио®',
 'Абакавир-Эдвансд',
 'Абактал® (Abaktal®)',
 'Абилифай (Abilify)',
 'Абиратерон (Abiraterone)',
 'Абиратерон НВ',
 'Абиратерон-ТЛ (Abiraterone-TL)',
 'Абисиб (Abisib)']

In [29]:
molec[:10]

['Дигидротахистерол* (Dihydrotachysterol*)',
 'A11CC02 Дигидротахистерол',
 'Регулятор обмена кальция и фосфора [Корректоры метаболизма костной и хрящевой ткани]',
 'Витамины и витаминоподобные средства',
 'Вакцина для профилактики дифтерии, столбняка, коклюша (бесклеточная), гепатита B и инфекции, вызываемой Haemophilus influenzae тип b (Vaccinum ad prophylaxim diphtheriae, tetani, pertussis, hepatitidis epidemicae B et infectionibus Haemophilus influenzae b)',
 'J07CA Вакцины бактериальные и вирусные в комбинации',
 'Вакцины, сыворотки, фаги и анатоксины',
 'Терифлуномид* (Teriflunomide*)',
 'L04AA31 Терифлуномид',
 'Иммунодепрессанты']

In [32]:
#list_to_file('rlsnet_drug_names.txt', names, 'utf-8')
#list_to_file('rlsnet_molec_names.txt', molec, 'utf-8')

## Парсинг БЗ Хеликс

In [2]:
#Сбор ссылок названий лабораторных тестов
main_page_str = "https://helix.ru"
main_page = req.get(main_page_str)

start_page = req.get("https://helix.ru/kb")
soup = BeautifulSoup(start_page.text, 'lxml')

tests = soup.find_all('a', attrs={ 'class' : 'Kb-Sections-Item'})
tests_links=[main_page_str+item.get('href') for item in tests]

In [3]:
#Сборс сылок названий заболеваний
disease=codecs.open("disease.html",'r','utf-8')
document= BeautifulSoup(disease.read(), 'html.parser')
disease_links = [item.get('href') for item in document.find_all('a')]
disease_links = [link for link in disease_links if link]
disease_links = [link for link in disease_links if 'item' in link]

In [4]:
#Сбор ссылок терминов
terms=codecs.open("terms.html",'r','utf-8')
document= BeautifulSoup(terms.read(), 'html.parser')
terms_links = [item.get('href') for item in document.find_all('a')]
terms_links = [link for link in terms_links if link]
terms_links = [link for link in terms_links if 'dict' in link]
all_links = tests_links+disease_links+terms_links

In [7]:
all_links = list(unique_everseen(all_links))
all_links = [item for item in all_links if 'tel' not in item]
len(all_links)

2381

In [8]:
#Сбор текстов по темам
texts=[]
for link in tq(all_links):
    page = req.get(link)
    soup = BeautifulSoup(page.text, 'lxml')
    content = soup.find_all('div', attrs={ 'class' : 'Kb-Content-Item-Content'})
    if content!=[]:
        c = content[0].get_text()
        if 'Литература' in content[0].get_text():
            ind = c.index('Литература')
            text = c[:ind]
        else:
            text = c
        text = text.replace('\n',' ').replace('\xa0',' ').replace('\ufeff',' ')
        text = re.sub(r'\s+', ' ', text)
        texts.append(text)


In [9]:
print('Количество текстов:', len(texts))

Количество текстов: 2215


In [10]:
t='\n'.join(texts)

In [11]:
#list_to_file('helix_bz.txt', texts, 'utf-8')